In [ ]:
import pygad 
import numpy as np 
from scipy import linalg 
from solver_for_Pocklington_equation import directional_pattern
import matplotlib.pyplot as plt

light_speed, mu0, eps0 = 299792458., 4*np.pi*1e-7, 8.854e-12
frequency = 1e6 * 146
omega = 2 * np.pi * frequency
incident_voltage = 10

delta_z = light_speed / frequency / 1e2

P_0 = 0.4

k_form, k_p = 1, 1
phi = np.linspace(1e-6, 2*np.pi-1e-6, 1000)
P = []
fit = []
def aim_func(phi):
    f = np.zeros(len(phi))
    for i in range (len(f)):
        f[i] = max(0,np.sin(np.pi/2 - phi[i]))**7
    return f


datas = []

def fitness_func(ga_instance, solution, solution_idx):
    
    generation = ga_instance.generations_completed
    
    element_position = np.zeros((num_elements, 3)) 
    element_length = np.zeros(num_elements)           
    for i in range(num_elements):
        element_position[i] = [0, solution[i], 0] 
        element_length[i] = solution[num_elements + i]     
    wire_radius = np.full(num_elements, 3.175e-3)
    source_position = np.array([element_position[0]], dtype=float) 
    
    for i in range(len(element_position)-1):
        for j in range(i+1,len(element_position)): 
            if(abs(element_position[i][1] - element_position[j][1]) <= 10*max(wire_radius[i],wire_radius[j])):
                fitness = 0
                print(f'Fitness: {fitness}, Generation: {generation}, Parent Index: {solution_idx}')
                return 0
            
    R, _, _, _, _, current, _, P_total, phi = directional_pattern (frequency, delta_z, incident_voltage, element_position, element_length, wire_radius, source_position)
    
    data = {
        'element_position': element_position,
        'element_length': element_length,
        'wire_radius': wire_radius,
        'source_position': source_position,
        'currents': current,
        'R': R
    }
    datas.append(data)
    
    fit_form = np.dot(aim_func(phi), P_total/np.max(P_total)) / np.linalg.norm(aim_func(phi)) / np.linalg.norm(P_total/np.max(P_total))
    fit_p = np.clip(np.exp(P_total[0]/P_0) - 1, 0, 1)
    fitness = k_form * fit_form + k_p * fit_p
    plt.polar(phi, P_total)
    print(f'Fitness: {fitness}, Generation: {generation}, Parent Index: {solution_idx}')
    P.append(P_total)
    fit.append(fitness)
    return fitness

num_elements = 8

num_generations = 2
num_parents_mating = 2
sol_per_pop = 10
num_genes = 2 * num_elements  

min_pos, max_pos = 0.0, 1.5 * light_speed / frequency
min_length, max_length = light_speed / frequency * 0.4, light_speed / frequency * 0.55
position_limit = [min_pos, max_pos]  
length_limit = [min_length, max_length]  

gene_space = []

for _ in range(num_elements):
    gene_space.append(position_limit)
for _ in range(num_elements):
    gene_space.append(length_limit)
    

initial_population = np.random.uniform(low=[g[0] for g in gene_space], high=[g[1] for g in gene_space], size=(sol_per_pop, num_genes))

def mutation_func(offspring, ga_instance):
    
    for chromosome_idx in range(offspring.shape[0]):
        
        chance_of_mutation = np.random.uniform(-1,1)
        fitness_values = ga_instance.last_generation_fitness 
        fitness_value = fitness_values[chromosome_idx]
        if fitness_value >= 1.9:
            for gene_idx in range(offspring.shape[1]):
                chance_of_mutation = np.random.uniform(-1,1)
                if chance_of_mutation >= 0.5 :
                    if gene_idx < num_elements:
                        mutation_value = np.random.uniform(-max_pos/70, max_pos/70) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_pos, max_pos)
                    else :
                        mutation_value = np.random.uniform(-max_length/70, max_length/70) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_length, max_length)
                        
        elif fitness_value >= 1.7:
            for gene_idx in range(offspring.shape[1]):
                chance_of_mutation = np.random.uniform(-1,1)
                if chance_of_mutation >= 0.8 :
                    if gene_idx < num_elements:
                        mutation_value = np.random.uniform(-max_pos/70, max_pos/70) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_pos, max_pos)
                    else :
                        mutation_value = np.random.uniform(-max_length/70, max_length/70) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_length, max_length)  
                        
        elif fitness_value >= 1.5:
            for gene_idx in range(offspring.shape[1]):
                chance_of_mutation = np.random.uniform(-1,1)
                if chance_of_mutation >= 0.9 :
                    if gene_idx < num_elements:
                        mutation_value = np.random.uniform(-max_pos/64, max_pos/64) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_pos, max_pos)
                    else :
                        mutation_value = np.random.uniform(-max_length/64, max_length/64) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_length, max_length)  
                        
        elif fitness_value >= 1.3:
            for gene_idx in range(offspring.shape[1]):
                chance_of_mutation = np.random.uniform(-1,1)
                if chance_of_mutation >= 0.75 :
                    if gene_idx < num_elements:
                        mutation_value = np.random.uniform(-max_pos/32, max_pos/32) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_pos, max_pos)
                    else :
                        mutation_value = np.random.uniform(-max_length/32, max_length/32) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_length, max_length)
                if chance_of_mutation <= -0.9 :
                    if gene_idx < num_elements:
                        mutation_value = np.random.uniform(-max_pos/10, max_pos/10) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_pos, max_pos)

                            
        elif fitness_value >= 1.1:
            for gene_idx in range(offspring.shape[1]):
                chance_of_mutation = np.random.uniform(-1,1)
                if chance_of_mutation >= 0.7 :
                    if gene_idx < num_elements:
                        mutation_value = np.random.uniform(-max_pos/16, max_pos/16) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_pos, max_pos)
                    else :
                        mutation_value = np.random.uniform(-max_length/16, max_length/16) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_length, max_length)
                if chance_of_mutation <= -0.7 :
                    if gene_idx < num_elements:
                        mutation_value = np.random.uniform(-max_pos/10, max_pos/10) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_pos, max_pos)

        elif fitness_value >= 0.9:
            for gene_idx in range(offspring.shape[1]):
                chance_of_mutation = np.random.uniform(-1,1)
                if chance_of_mutation >= 0.7 :
                    if gene_idx < num_elements:
                        mutation_value = np.random.uniform(-max_pos/8, max_pos/8) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_pos, max_pos)
                    else :
                        mutation_value = np.random.uniform(-max_length/8, max_length/8) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_length, max_length)
                elif chance_of_mutation <= -0.95 :
                    if gene_idx < num_elements:
                        mutation_value = np.random.uniform(-max_pos/4, max_pos/4) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_pos, max_pos)
                                              
        elif fitness_value >= 0.70:
            for gene_idx in range(offspring.shape[1]):
                chance_of_mutation = np.random.uniform(-1,1)
                if chance_of_mutation >= 0.7 :
                    if gene_idx < num_elements:
                        mutation_value = np.random.uniform(-max_pos/4, max_pos/4) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_pos, max_pos)
                    else :
                        mutation_value = np.random.uniform(-max_length/4, max_length/4) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_length, max_length)
                        
        elif fitness_value < 0.70:
            for gene_idx in range(offspring.shape[1]):
                chance_of_mutation = np.random.uniform(-1,1)
                if chance_of_mutation >= 0.65 :
                    if gene_idx < num_elements:
                        mutation_value = np.random.uniform(-max_pos/2, max_pos/2) 
                        offspring[chromosome_idx, gene_idx] += mutation_value
                        offspring[chromosome_idx, gene_idx] = np.clip(offspring[chromosome_idx, gene_idx], min_pos, max_pos)
                        
    return offspring

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_func,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       initial_population=initial_population,
                       gene_space=gene_space,
                       mutation_type=mutation_func,
                       parent_selection_type='tournament',
                       crossover_type='single_point') 

ga_instance.run()

solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Лучшее решение:", solution) 
print("Оценка лучшего решения:", solution_fitness) 

datas = np.array(datas)
np.save('data.npy', datas)

In [ ]:
delta_z = light_speed / frequency / 5e2
element_position = np.zeros((num_elements, 3)) 
element_length = np.zeros(num_elements)           
for i in range(num_elements):
    element_position[i] = [0, solution[i], 0] 
    element_length[i] = solution[num_elements + i]     
wire_radius = np.full(num_elements, 3.175e-3)
source_position = np.array([element_position[0]], dtype=float) 
R, R_block, element_num, incident_field, impedance, current, current_block, P_total, phi = directional_pattern (frequency, delta_z, incident_voltage, element_position, element_length, wire_radius, source_position)
fitness = np.dot(aim_func(phi), P_total) / np.linalg.norm(aim_func(phi)) / np.linalg.norm(P_total)
plt.polar(phi, P_total/np.max(P_total), label = f'fitness: {np.round(solution_fitness,3)}')
plt.polar(phi,aim_func(phi))
plt.legend()
plt.show()

In [ ]:
plt.polar(phi, P_total, label = f'fitness: {np.round(solution_fitness,3)}')
plt.polar(phi,aim_func(phi)*P_0)
plt.legend()
plt.show()

In [ ]:
plt.plot(phi, P_total)
plt.plot(phi,aim_func(phi)*P_0)

In [ ]:
plt.plot(np.arange(len(fit)),fit)
plt.title('Changing of fitness during genetic algorithm', size = 14)
plt.ylabel('Fitness of solution', size = 14)
plt.xlabel('Number of configuration', size = 14)

In [ ]:
from visualization import plot_2dmodel, plot_together, plot_separately, current_distribution_2d, current_distribution_3d

plot_2dmodel(R, source_position, sum(element_num), delta_z)
#plot_together(R_block, current_block)
#plot_separately(R_block, current_block)
current_distribution_2d(R, current)
#current_distribution_3d(R, source_position, R_block, current_block)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio

num_frames = len(P)

frames = []

for i in range(num_frames):
    plt.figure(figsize=(6, 6))
    
    plt.polar(phi, P[i], label = f'fitness: {np.round(fit[i],3)}')  
    plt.polar(phi,aim_func(phi)*P_0)
    plt.legend()
    plt.title(f"Configuration {i + 1}")
    
    plt.savefig('temp_frame.png')
    plt.close()  
    
    frames.append(imageio.imread('temp_frame.png'))

imageio.mimsave('evalution_of_optimizator.gif', frames, duration=1.5)  



In [ ]:
import numpy as np
import imageio

# Загрузка списка словарей из файла .npy
loaded_dicts = np.load('data.npy', allow_pickle=True)

# Список для хранения кадров анимации
frames = []
def plot_2dmodel (R, source_position, num_elements, delta_z):

    #* Create figure and axes
    fig, ax = plt.subplots()
    plt.errorbar(np.array(R[:,1]),np.array(R[:,2]), xerr = delta_z, linestyle = "none", marker = '.', markersize = '2', zorder = 10, color = 'blue')
    for i in range (len(R)):
        for k in range (len(source_position)):
            if all(source_position[k] == R[i]):
                plt.errorbar(np.array(R[i,1]),np.array(R[i,2]), xerr = delta_z, linestyle = "none", marker = '.', markersize = '5', zorder = np.inf, color = 'red')
    #* Set limits for axes 
    ax.set_xlim(min(R[:,1])-0.1, max(R[:,1])+0.1)
    ax.set_ylim(min(R[:,2])-0.1, max(R[:,2]+0.1))

    #* Name the axes and graph
    ax.set_title('2D Model of Yagi-Uda Antenna', size = 14)
    ax.set_xlabel('Y position, m', size = 12)
    ax.set_ylabel('Z position, m', size = 12)

    #* Add grid
    ax.grid(True)

    #* show figure
    plt.show()
    
for i in range(len(loaded_dicts)):
    
    R = loaded_dicts[i]['R']
    source_position = loaded_dicts[i]['source_position']
    fig, ax = plt.subplots()
    plt.errorbar(np.array(R[:,1]),np.array(R[:,2]), xerr = delta_z, linestyle = "none", marker = '.', markersize = '2', zorder = 10, color = 'blue')
    for i in range (len(R)):
        for k in range (len(source_position)):
            if all(source_position[k] == R[i]):
                plt.errorbar(np.array(R[i,1]),np.array(R[i,2]), xerr = delta_z, linestyle = "none", marker = '.', markersize = '5', zorder = np.inf, color = 'red')
    #* Set limits for axes 
    ax.set_xlim(min(R[:,1])-0.1, max(R[:,1])+0.1)
    ax.set_ylim(min(R[:,2])-0.1, max(R[:,2]+0.1))

    #* Name the axes and graph
    ax.set_title('2D Model of Yagi-Uda Antenna', size = 14)
    ax.set_xlabel('Y position, m', size = 12)
    ax.set_ylabel('Z position, m', size = 12)
    
    plt.savefig('temp_frame.png')
    plt.close() 
    
    frames.append(imageio.imread('temp_frame.png'))

imageio.mimsave('evalution_of_positions.gif', frames, duration=0.7)  

